### Backtesting Statistics

#### 일반적인 특성

다음 통계량은 백테스트의 일반적인 특성을 알려 준다

- Time range : 시작 날짜와 종료 날짜를 지정한다. 전략을 테스트하는데 사용되는 기간이 충분히 길어 포괄적인 상황을 포함할 수 있어야 한다
- Average AUM : 운용 중인 자산의 평균 달러 가치다. 이 평균을 계산하고자 매수와 매도 포지션의 달러 가치를 양의 실수로 간주한다
- Capacity : 전략의 Capacity는 목표 리스크 조정 성과를 성취할 수 있는 최대 AUM으로 측정할 수 있다. 적절한 베팅 크기와 리스크 분산을 위해서는 최저 AUM이 필요하다. 최소 AUM을 넘어서면 AUM의 증가에 따라 더 높은 거래 비용과 더 낮은 거래량으로 성과가 저하된다
- Leverage : 보고된 성과를 달성하고자 필요한 차입 총액을 측정한다. 레버리지가 일어나면 비용이 할당되어야 한다. 레버리지를 측정하는 한 가지 방법은 평균 AUM에 대한 평균 달러 포지션 크기의 비율을 계산하는 것이다
- Maximum dollar position size : 전략이 어느 시점에 평균 AUM을 크게 초과하는 달러 포지션을 취하는지 알려 준다. 대개 평균 AUM에 근접한 최대 달러 포지션을 취하는 전략을 선호한다. 이는 극단적인 이벤트에 의존하지 않았음을 알려 준다
- Ratio of longs : 
- Frequency of bets : 
- Average holding period :
- Annualized turnover :
- Correlation to underlying :

In [1]:
import numpy as np
import pandas as pd

import yfinance as yf

start_date = '1962-01-01'
end_date = '2024-01-01'

ibm = yf.download('IBM', start = start_date, end = end_date)
ibm_ret = np.log(ibm.Close / ibm.Close.shift(1)).dropna()
ibm_ret.head()

[*********************100%%**********************]  1 of 1 completed


Date
1962-01-03    0.008703
1962-01-04   -0.010015
1962-01-05   -0.019890
1962-01-08   -0.018928
1962-01-09    0.011760
Name: Close, dtype: float64

In [2]:
from FinancialMachineLearning.backtest.backtest_statistics import *

tpos = timing_of_flattening_and_flips(target_positions = ibm_ret)

In [3]:
ibm_ret[tpos]

Date
1962-01-04   -0.010015
1962-01-09    0.011760
1962-01-16   -0.010648
1962-01-18    0.002716
1962-01-22   -0.001808
                ...   
2023-12-13   -0.006640
2023-12-18    0.003139
2023-12-19   -0.007277
2023-12-21    0.004551
2023-12-29   -0.001222
Name: Close, Length: 7824, dtype: float64

In [4]:
average_holding_period(ibm_ret)

1.9722818971740315

In [5]:
average_holding_period(ibm_ret[tpos])

2.7516034013053217

#### Run

투자 전략은 IID process로부터 추출된 수익률을 생성하는 경우가 드물다

**수익률 집중**

In [6]:
rHHIPos = bets_concentration(ibm_ret[ibm_ret >= 0])
rHHINeg = bets_concentration(ibm_ret[ibm_ret < 0])

In [7]:
print(f"positive return concentration : {rHHIPos}")
print(f"negative return concentration : {rHHINeg}")

positive return concentration : 0.0001344589493353432
negative return concentration : 0.0001434386472541928


In [8]:
tHHI = bets_concentration(
    ibm_ret.groupby(pd.Grouper(freq='M')).count()
)

In [9]:
tHHI

4.558249213681642e-06

**Drawdown and Time under Water**

In [10]:
drawdown, time_under_water = drawdown_and_time_under_water(ibm_ret)